# Resting Heart Rate Algorithm Comparison (Fitbit Intraday Data)

In this example, two simple resting heart rate algorithms are computed using Fitbit intraday data:
* Sleeping resting heart rate (median intraday heart rate during Fitbit sleep intervals)
* Inactivity resting heart rate (heart rate when, based on intraday step data, participant has not moved for 20 minutes)

The results of these algorithms are compared to the resting heart rate value calculated by the Fitbit device.

#### All Outputs Saved In This Notebook Are Sample/Demo Data

Clear all outputs before running this notebook.

### Gather Your Export Database Configuration Settings and Credentials

To obtain the configuration settings for your project"s Export Database, open MyDataHelps Designer and navigate to the `Settings` tab for your project. Click `Export Explorer`. The `External Applications` tab will provide the required configuration settings. 

In [ ]:
import os, sys; sys.path.append("..") 
from src.mdh_query import MDHQuery

import pandas as pd
import numpy as np

mdh_query = MDHQuery(
    catalog = "{YOUR CATALOG}",
    workgroup = "{YOUR WORKGROUP}",
    s3_output_location = "{YOUR S3 OUTPUT LOCATION}",
    aws_profile_name = "{YOUR AWS PROFILE NAME}" 
)

### Fitbit Native RHR

For comparison with our custom algorithms, the daily resting heart rate that is calculated by Fitbit is loaded into a data frame.

In [19]:
participant_identifier = "{PARTICIPANT IDENTIFIER}"

In [20]:
query_string = f"""
select 
    cast(date as date) as sample_date, 
    restingheartrate as precomputed_rhr 
from
    fitbitdailydata
where 
    participantidentifier = '{participant_identifier}'
order by
    date desc
"""

precomputed_rhr = mdh_query.get_query_result(query_string)
precomputed_rhr.head(5)

11:43:59 query status: QUEUED
11:44:04 query status: SUCCEEDED
11:44:05 rows: 152 columns: ['sample_date', 'precomputed_rhr']


,sample_date,precomputed_rhr
0,2022-10-16,56.0
1,2022-10-15,55.0
2,2022-10-14,56.0
3,2022-10-13,54.0
4,2022-10-12,53.0


### Super Simple Nighttime RHR

A super simple resting heart rate is computed from intraday heart rate data (one data point per second). In this example, we obtain the median heart rate between the hours of midnight and 5am.

In [22]:
query_string = f"""
select 
    cast(datetime as date) as sample_date, 
    count(*) as intraday_instances,
    min(value) as min,
    max(value) as max,
    avg(value) as avg,
    approx_percentile(value,0.25) as pct25,
    approx_percentile(value,0.5) as median,
    approx_percentile(value,0.75) as pct75
from 
    fitbitintradaycombined
where 
    type = 'activities-heart'
    and hour(cast(datetime as timestamp)) < 5
    and participantidentifier = '{participant_identifier}'
group by 
    1
order by 
    1 desc
"""

super_simple_intraday_rhr = mdh_query.get_query_result(query_string)
super_simple_intraday_rhr.head(5)    

11:44:25 query status: QUEUED
11:44:30 query status: SUCCEEDED
11:44:30 rows: 138 columns: ['sample_date', 'intraday_instances', 'min', 'max', 'avg', 'pct25', 'median', 'pct75']


,sample_date,intraday_instances,min,max,avg,pct25,median,pct75
0,2022-10-12,2212,43.0,84.0,49.952080,47.0,49.0,51.0
1,2022-10-11,2230,42.0,80.0,48.795067,46.0,48.0,50.0
2,2022-10-10,2128,43.0,74.0,47.406015,45.0,46.0,48.0
3,2022-10-09,2161,41.0,78.0,47.799630,45.0,47.0,49.0
4,2022-10-08,2234,43.0,79.0,49.937332,47.0,49.0,51.0


### Simple Sleeping RHR

A simple resting heart rate (although somewhat improved on the fixed time interval approach used above) is computed from intraday heart rate data (one data point per second). In this example, sleep intervals are obtained from the Fitbit sleep log data. Intraday heart rates observed during sleep intervals us used to compute a simple nighttime RHR.

In [43]:
query_string = f"""
with sleep_intervals as (	
select 
    sleeplogdate,
    startdate,
    enddate,
    value
from
    fitbitsleeplogdetails
where 
    participantidentifier = '{participant_identifier}'
    and value in ('rem', 'light', 'deep')
order by
    sleeplogdate desc
)
select 
    cast(sl.sleeplogdate as date) as sample_date, 
    count(*) as intraday_instances, 
    approx_percentile(hr.value,0.25) as sleeping_rhr_pct25,
    approx_percentile(hr.value,0.5) as sleeping_rhr,
    approx_percentile(hr.value,0.75) as sleeping_rhr_pct75
from 
    fitbitintradaycombined hr
    join sleep_intervals sl on sl.startdate <= hr.datetime and sl.enddate >= hr.datetime
where 
    type = 'activities-heart'
    and participantidentifier = '{participant_identifier}'
group by 1
order by 1 desc 
"""

sleeping_rhr = mdh_query.get_query_result(query_string)
sleeping_rhr.head(5)    

16:26:00 query status: QUEUED
16:26:05 query status: RUNNING
16:26:10 query status: SUCCEEDED
16:26:10 rows: 133 columns: ['sample_date', 'intraday_instances', 'sleeping_rhr_pct25', 'sleeping_rhr', 'sleeping_rhr_pct75']


,sample_date,intraday_instances,sleeping_rhr_pct25,sleeping_rhr,sleeping_rhr_pct75
0,2022-10-12,3049,47.0,49.0,50.0
1,2022-10-11,3136,46.0,49.0,52.0
2,2022-10-10,3164,45.0,46.0,48.0
3,2022-10-09,3753,45.0,47.0,50.0
4,2022-10-08,3690,47.0,49.0,52.0


### Inactivity RHR

Fitbit intraday step data is used to determine periods where a participant is inactive (zero steps) for 20 minutes or longer.

Mean intraday heart rate is determined for these periods.

In [25]:
query_string = f"""
select 
    participantidentifier,
    datetime as sample_date 
from 
    fitbitintradaycombined hr
where 
    participantidentifier = '{participant_identifier}'
    and type = 'activities-steps'
    and datetime > date_parse('01-01-22', '%m-%d-%y')
    and hour(cast(datetime as timestamp)) >= 9
    and hour(cast(datetime as timestamp)) <= 21
    and value = 0
order by
    datetime
"""

inactivity = mdh_query.get_query_result(query_string)

inactivity["sample_date"] = pd.to_datetime(inactivity["sample_date"])
series = inactivity.groupby("participantidentifier").sample_date.diff().dt.seconds.ne(60).cumsum()

inactivity_intervals = (inactivity.groupby(["participantidentifier", series])
   .agg(start=("sample_date", 'min'), end=("sample_date", 'max'))
   .droplevel(1)
   .reset_index()
)
inactivity_intervals["duration"] = (inactivity_intervals["end"] - inactivity_intervals["start"]).astype('timedelta64[m]')
inactivity_intervals["sample_date"] = inactivity_intervals["start"].dt.date
inactivity_intervals["start_rhr"] = inactivity_intervals["start"] + pd.Timedelta(minutes=5)
inactivity_intervals[inactivity_intervals.duration >= 20].tail(5)

15:52:49 query status: QUEUED
15:52:55 query status: SUCCEEDED
15:52:56 rows: 86204 columns: ['participantidentifier', 'sample_date']


,participantidentifier,start,end,duration,sample_date,start_rhr
16306,MDH-1514-7516,2022-10-12 13:05:00,2022-10-12 13:36:00,31.0,2022-10-12,2022-10-12 13:10:00
16309,MDH-1514-7516,2022-10-12 13:56:00,2022-10-12 14:59:00,63.0,2022-10-12,2022-10-12 14:01:00
16310,MDH-1514-7516,2022-10-12 15:01:00,2022-10-12 15:27:00,26.0,2022-10-12,2022-10-12 15:06:00
16320,MDH-1514-7516,2022-10-12 17:53:00,2022-10-12 18:18:00,25.0,2022-10-12,2022-10-12 17:58:00
16335,MDH-1514-7516,2022-10-12 20:35:00,2022-10-12 21:59:00,84.0,2022-10-12,2022-10-12 20:40:00


In [33]:
query_string = f"""
select 
    participantidentifier,
    datetime as observed_time,
    value
from 
    fitbitintradaycombined hr
where 
    participantidentifier = '{participant_identifier}'
    and type = 'activities-heart'
    and datetime > date_parse('01-01-22', '%m-%d-%y')
    and hour(cast(datetime as timestamp)) >= 9
    and hour(cast(datetime as timestamp)) <= 21
order by
    datetime
"""

all_intraday_hr = mdh_query.get_query_result(query_string)
all_intraday_hr["observed_time"] = pd.to_datetime(all_intraday_hr["observed_time"])

a = all_intraday_hr.observed_time.values
bh = inactivity_intervals.end.values
bl = inactivity_intervals.start_rhr.values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

intraday_hr_during_inactivity = pd.concat([
    all_intraday_hr.loc[i, :].reset_index(drop=True),
    inactivity_intervals.loc[j, :].reset_index(drop=True)
], axis=1)

inactivity_rhr = (intraday_hr_during_inactivity.groupby("sample_date")   
    .agg(inactivity_rhr=("value", 'mean'))
    .reset_index()
)
inactivity_rhr.head(5)

15:55:10 query status: QUEUED
15:55:15 query status: RUNNING
15:55:20 query status: SUCCEEDED
15:55:28 rows: 1026462 columns: ['participantidentifier', 'observed_time', 'value']


,sample_date,inactivity_rhr
0,2022-05-21,66.403427
1,2022-05-22,59.814978
2,2022-05-23,59.820579
3,2022-05-24,62.419518
4,2022-05-25,56.533333


### Compare Native RHR to Computed RHRs

In [42]:
sleeping_rhr.sample_date = pd.to_datetime(sleeping_rhr.sample_date)
precomputed_rhr.sample_date = pd.to_datetime(precomputed_rhr.sample_date)
inactivity_rhr.sample_date = pd.to_datetime(inactivity_rhr.sample_date)

rhr = pd.merge(precomputed_rhr, sleeping_rhr[["sample_date", "sleeping_rhr"]], how="outer", on="sample_date")
rhr = pd.merge(rhr, inactivity_rhr, how="outer", on="sample_date")
rhr["diff_precomputed_sleep"] = rhr["precomputed_rhr"] - rhr["sleeping_rhr"]
rhr["diff_precomputed_inactivity"] = rhr["precomputed_rhr"] - rhr["inactivity_rhr"]
rhr[rhr.sleeping_rhr.notna() & rhr.inactivity_rhr.notna()].head(5)

,sample_date,precomputed_rhr,sleeping_rhr,inactivity_rhr,diff_precomputed_sleep,diff_precomputed_inactivity
4,2022-10-12,53.0,49.0,61.725527,4.0,-8.725527
5,2022-10-11,53.0,49.0,63.571928,4.0,-10.571928
6,2022-10-10,52.0,46.0,54.140567,6.0,-2.140567
8,2022-10-08,53.0,49.0,56.311475,4.0,-3.311475
9,2022-10-07,52.0,49.0,59.238555,3.0,-7.238555
